In [ ]:
import csv
import os
from neo4j import GraphDatabase
from predibase import PredibaseClient

In [ ]:
neo4j_uri = os.environ.get('NEO4J_URI')
neo4j_username = os.environ.get('NEO4J_USERNAME')
neo4j_password = os.environ.get('NEO4J_PASSWORD')
neo4j_auth =  (neo4j_username, neo4j_password)

In [ ]:
pc = PredibaseClient()
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_auth))

In [ ]:
# helper function to execute the predibase driver

def execute_query(driver, query):
    with driver.session() as session:
        def _execute(tx):
            result = tx.run(query)
            return [record for record in result]
        return session.execute_read(_execute)

#### Test queries in CSV on DB, sort into status

In [ ]:

def save_to_csv(filename, data, headers):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)

In [ ]:
def execute_and_classify_queries(input_csv, driver):
    queries_with_errors = []
    queries_with_no_data = []
    queries_with_data = []

    with open(input_csv, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            query = row['Output']  # Assuming 'Output' column has the queries
            try:
                result = execute_query(driver, query)
                if result:
                    queries_with_data.append((row['Input'], query, result))
                else:
                    queries_with_no_data.append((row['Input'], query))
            except Exception as e:
                queries_with_errors.append((row['Input'], query, str(e)))

    # Save to CSVs
    save_to_csv('error.csv', queries_with_errors, ['Question', 'Query', 'Error'])
    save_to_csv('no_data.csv', queries_with_no_data, ['Question', 'Query'])
    save_to_csv('clean.csv', queries_with_data, ['Question', 'Query', 'Result'])

In [ ]:
# Replace 'your_input_csv.csv' with the path to your input CSV file containing questions and queries
execute_and_classify_queries('your_input_csv.csv', driver)